In [1]:
from nltk.corpus import wordnet as wn
import xlwt

In [ ]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict, Counter

In [9]:
word = 'red.n.01'
s = wn.synset(word)
print(s.definition())
print(s.hyponyms())

red color or pigment; the chromatic color resembling the hue of blood
[Synset('cardinal.n.03'), Synset('cerise.n.01'), Synset('chrome_red.n.01'), Synset('crimson.n.01'), Synset('dark_red.n.01'), Synset('purplish_red.n.01'), Synset('sanguine.n.01'), Synset('scarlet.n.01'), Synset('turkey_red.n.01')]


In [11]:
n = 0
off = []
for line in open("alignment.tsv",encoding = "UTF-8"):
    l = line.split('\t')
    offset = int(l[0])
    if offset not in off:
        off.append(offset)

KeyboardInterrupt: 

In [ ]:
print(len(off))

In [4]:
print(wn.synsets('red'))

[Synset('red.n.01'), Synset('red.n.02'), Synset('bolshevik.n.01'), Synset('loss.n.06'), Synset('red.s.01'), Synset('crimson.s.02'), Synset('crimson.s.03')]


In [ ]:
corpus = []

In [ ]:
import re, string
table = {ord(c): None for c in string.punctuation}

In [ ]:
hypernyms = []

for h in s.hypernyms():
    temp = []
    hd = h.definition().split()
    for h in hd:
        temp.append(h.translate(table))
    definition = " ".join(temp)
    hypernyms.append(definition)

document1 = " ".join(hypernyms)
print(document1)

In [ ]:
hyponyms = []

for h in s.hyponyms():
    temp = []
    hd = h.definition().split()
    for h in hd:
        temp.append(h.translate(table))
    definition = " ".join(temp)
    hyponyms.append(definition)
    
document2 = " ".join(hyponyms)
print(document2)

In [ ]:
corpus.append(document1)
corpus.append(document2)

print(corpus)

In [ ]:
tf_idf_Dict = defaultdict(lambda: list())
vectorizer = TfidfVectorizer()
try:
    print(corpus)
    tfidf = vectorizer.fit_transform(corpus)
    print(tfidf.shape)

    words = vectorizer.get_feature_names()
    #print(words)
    for i in range(len(corpus)):
        print('----Document %d----' % (i))
        for j in range(len(words)):
            print("words[j] ", words[j]," tfidf ", tfidf[i,j])
            if tfidf[i,j] > 0.25:
                tf_idf_Dict[word].append(words[j])
except:
    print(word,'x')

In [ ]:
print(tf_idf_Dict[word])

In [ ]:
d = s.definition()
de = d.split()

In [ ]:
definition = []
for d in de:
    definition.append(d.translate(table))
    
print(definition)

In [ ]:
for d in definition:
    if d in tf_idf_Dict[word]:
        print(d)

### test

In [ ]:
from collections import defaultdict,Counter

In [530]:
ss = wn.synset('right.n.01')
offset = str(ss.offset())
print(offset)

5174653


In [490]:
wn.synset_from_pos_and_offset('n', 7140659)

Synset('discussion.n.02')

### word2vec ###

In [227]:
from gensim.models import KeyedVectors
import logging
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [389]:
from gensim import models
from gensim.models import word2vec

#### save model ####

In [281]:
model.save("word2vec_withgooglenews")

In [ ]:
print(model.most_similar('plant', topn = 3))

In [ ]:
w = 'plant'
#print(model.most_similar(positive=['plant','flora'], topn = 10))
for word in model.most_similar(positive=['plant','flora'], topn = 10):
    if w not in str(word[0]).lower():
        print(str(word[0]).lower())

In [ ]:
print(model.most_similar(positive=['plant','flora','botany'], topn = 4))

In [ ]:
print(model.most_similar(positive=['plant','organism','crop'], topn = 4))

### alignment

In [461]:
threshold1 = 0.15
threshold2 = 0.55

In [462]:
temp_alignment = defaultdict(lambda:list())
for line in open("alignment.tsv",encoding = "UTF-8"):
    l = line.split('\t')
    offset = int(l[0])
    title = l[1].lower()
    p1 = float(l[2])
    p2 = float(l[3])
    temp_alignment[offset].append('')
    if(p1 < threshold1 and p2 < threshold2):continue
    index1 = title.find('(')
    if index1 > 0:
        index2 = title.find(')')
        title = title[index1+1:index2]
        #print(title)
    if title not in model.wv.vocab:continue
    temp_alignment[offset].append(title)
    #print(offset,title)



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [503]:
print(temp_alignment[17222])

['botany', 'flora', 'plant']


In [463]:
for offset in temp_alignment.keys():
    if('' in temp_alignment[offset]):
        temp_alignment[offset] = list(filter(lambda x: x != '', temp_alignment[offset]))

In [441]:
print(temp_alignment)

defaultdict(<function <lambda> at 0x126920730>, {3553: []})


In [464]:
alignment = defaultdict(lambda:list())
for offset in temp_alignment.keys():
    sys = wn.synset_from_pos_and_offset('n', offset)
    alignment[sys] = []
    for title in temp_alignment[offset]:
        s = sys.name().split('.')[0]
        #print(s)
        if str(s) not in str(title):
            alignment[sys].append(str(title))

In [443]:
print(alignment)

defaultdict(<function <lambda> at 0x1278b7840>, {Synset('whole.n.02'): []})


### lemma ###

In [465]:
lemma_dict = defaultdict(lambda:list())
for sys in alignment.keys():
    s = sys.name().split('.')[0]
    # Those lemma lower than 1 will be filter out
    #lemma_dict[sys] = []
    for lemma in sys.lemmas():
        l = str(lemma.name())
        if s not in l:
            if l not in model.wv.vocab:continue
            lemma_dict[sys].append(l)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


In [445]:
print(lemma_dict)


defaultdict(<function <lambda> at 0x126920a60>, {Synset('whole.n.02'): ['unit']})


### definition ###

In [466]:
definition_dict = defaultdict(lambda: '')
for sys in lemma_dict.keys():
    definition_dict[sys] = sys.definition()

In [473]:

print(definition_dict[])

AttributeError: 'str' object has no attribute '_name'

### hypernyms and hyponyms ###

In [467]:
hypernyms_dict = defaultdict(lambda:list())
for sys in lemma_dict.keys():
    hypernyms_dict[sys] = []
    for hy in sys.hypernyms():
        hyper = str(hy.name().split('.')[0])
        if hyper not in model.wv.vocab:continue
        hypernyms_dict[sys].append(hyper)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [449]:
print(hypernyms_dict[wn.synsets('plant')[1]])

[]


In [468]:
hyponyms_dict = defaultdict(lambda:list())
for sys in lemma_dict.keys():
    hyponyms_dict[sys] = []
    for ho in sys.hyponyms():
        hypo = str(ho.name().split('.')[0])
        if hypo not in model.wv.vocab:continue
        hyponyms_dict[sys].append(hypo)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [451]:
print(hyponyms_dict[wn.synsets('plant')[1]])

[]


### result ###

In [469]:
def searchInWord2Vec(input_word, input_list):
    result = []
    for word in model.most_similar(positive=input_list, topn = 5):
        if input_word not in str(word[0]).lower():
            result.append(str(word[0]).lower())
    return result

In [470]:
result_dict = defaultdict(lambda: list())
for sys in lemma_dict.keys():
    input_word = sys.name().split('.')[0]
    key = sys.name()
    if input_word not in model.wv.vocab:continue
    # combination 1 :lemma*1 + original word
    for lemma in lemma_dict[sys]:
        result_dict[key].append(['c1'] + searchInWord2Vec(input_word, [lemma] + [input_word]))
    # combination 2: lemma*2 + original word
    com2 = list(itertools.combinations(lemma_dict[sys],2))
    for c2 in com2:
        result_dict[key].append(['c2'] + searchInWord2Vec(input_word, list(c2) + [input_word]))
    # combination 3: lemma*3 + original word
    com3 = list(itertools.combinations(lemma_dict[sys],3))
    for c3 in com3:
        result_dict[key].append(['c3'] + searchInWord2Vec(input_word, list(c3) + [input_word]))
    # combination 4: hyper + lemma + original word
    for lemma in lemma_dict[sys]:
        for hy in hypernyms_dict[sys]:
            result_dict[key].append(['c4'] + searchInWord2Vec(input_word, [lemma] + [input_word] + [hy]))
    # combination 5: hyper + original word
    for hy in hypernyms_dict[sys]:
        result_dict[key].append(['c5'] + searchInWord2Vec(input_word, [input_word] + [hy]))
    # combination 6: hypo + lemma + original word
    for lemma in lemma_dict[sys]:
        for ho in hyponyms_dict[sys]:
            result_dict[key].append(['c6'] + searchInWord2Vec(input_word, [lemma] + [input_word] + [ho]))
    # combination 7: hypo + original word
    for ho in hyponyms_dict[sys]:
            result_dict[key].append(['c7'] + searchInWord2Vec(input_word, [input_word] + [ho]))
    # combination 8: wiki title*1 + original word
    for title in alignment[sys]:
        result_dict[key].append(['c8'] + searchInWord2Vec(input_word, [title] + [input_word]))
    # combination 9: wiki title*2 + original word
    com9 = list(itertools.combinations(alignment[sys],2))
    for c9 in com9:
        result_dict[key].append(['c9'] + searchInWord2Vec(input_word, list(c9) + [input_word]))
    # combination 10: wiki title*3 + original word
    com10 = list(itertools.combinations(alignment[sys],2))
    for c10 in com10:
        result_dict[key].append(['c10'] + searchInWord2Vec(input_word, list(c10) + [input_word]))
    # combination 11: wiki title + lemma + original word
    for lemma in lemma_dict[sys]:
        for title in alignment[sys]:
            result_dict[key].append(['c11'] + searchInWord2Vec(input_word, [lemma] + [input_word] + [title]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [471]:
import json
with open('hello.json','w') as fp:
    json.dump(result_dict,fp)

In [531]:
print(result_dict['plant.n.01'])

[['c1', 'factory', 'lime_kiln', 'bio_digester'], ['c6', 'factory', 'brewery', 'bottling', 'lime_kiln'], ['c6', 'distillery', 'factory', 'breweries'], ['c6', 'brewery', 'lime_kiln', 'distilleries'], ['c6', 'factories', 'manufacturing', 'toyota_bodine_aluminum'], ['c6', 'factory', 'hydroponic_tomato', 'unopened_buds', 'parsley_chervil'], ['c6', 'factory', 'canning_factory', 'cannery'], ['c6', 'bpd_refinery', 'refineries', 'refinery', 'motiva_refinery'], ['c6', 'lime_kiln', 'biodigester', 'bio_digester', 'papermill'], ['c6', 'aluminum_smelter', 'copper_smelter', 'smelting', 'lime_kiln'], ['c7', 'factory', 'brewery', 'bottling', 'distillery'], ['c7', 'distillery', 'factory', 'breweries'], ['c7', 'brewery', 'distilleries', 'whiskey_distillery'], ['c7', 'factories', 'manufacturing', 'mill'], ['c7', 'factory', 'unopened_buds', 'parsley_chervil'], ['c7', 'factory', 'cannery', 'peach_orchard'], ['c7', 'refineries', 'refinery', 'bpd_refinery', 'motiva_refinery'], ['c7', 'factory', 'lime_kiln', '

In [511]:
for sys in result_dict.keys():
    result = []
    for l in result_dict[l]:
        result

RuntimeError: dictionary changed size during iteration

In [304]:
import itertools
li = list(itertools.combinations([1,2,3,4],3))
for l in li:
    print(list(l))

[1, 2, 3]
[1, 2, 4]
[1, 3, 4]
[2, 3, 4]


In [376]:
for sys in alignment.keys():
    print('sys', sys)
    print('lemma', lemma_dict[sys])
    com2 = list(itertools.combinations(lemma_dict[sys],2))
    print(com2)
    for c in com2:
        print(list(c))
        print(searchInWord2Vec(input_word, list(c)))

sys Synset('abstraction.n.06')
lemma []
[]
sys Synset('thing.n.12')
lemma []
[]
sys Synset('object.n.01')
lemma []
[]
sys Synset('whole.n.02')
lemma ['unit']
[]
sys Synset('congener.n.03')
lemma []
[]
sys Synset('living_thing.n.01')
lemma []
[]
sys Synset('organism.n.01')
lemma ['being']
[]
sys Synset('benthos.n.02')
lemma []
[]
sys Synset('dwarf.n.03')
lemma []
[]
sys Synset('heterotroph.n.01')
lemma []
[]
sys Synset('parent.n.02')
lemma []
[]
sys Synset('life.n.10')
lemma []
[]
sys Synset('cell.n.02')
lemma []
[]
sys Synset('causal_agent.n.01')
lemma ['cause']
[]
sys Synset('person.n.01')
lemma ['individual', 'someone', 'somebody', 'mortal', 'soul']
[('individual', 'someone'), ('individual', 'somebody'), ('individual', 'mortal'), ('individual', 'soul'), ('someone', 'somebody'), ('someone', 'mortal'), ('someone', 'soul'), ('somebody', 'mortal'), ('somebody', 'soul'), ('mortal', 'soul')]
['individual', 'someone']
['somebody', 'person', 'somebody_else', 'anyone', 'individuals']
['indivi

In [387]:
word = 'abstraction'
s = wn.synsets(word)
for i in s:
    print(i.lemmas())

[Lemma('abstraction.n.01.abstraction'), Lemma('abstraction.n.01.abstract')]
[Lemma('abstraction.n.02.abstraction')]
[Lemma('abstraction.n.03.abstraction'), Lemma('abstraction.n.03.generalization'), Lemma('abstraction.n.03.generalisation')]
[Lemma('abstraction.n.04.abstraction')]
[Lemma('abstractedness.n.01.abstractedness'), Lemma('abstractedness.n.01.abstraction')]
[Lemma('abstraction.n.06.abstraction'), Lemma('abstraction.n.06.abstract_entity')]
